In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../")

import torch
import numpy as np
import matplotlib.pyplot as plt

import implicitmodules.torch as im

In [2]:
dim = 2
sigma = 2.
sigma1 = 2.
nb_pts = 2

it = 10

In [3]:
gd_trans = torch.tensor([[-2., 0.], [2., 0.]], requires_grad=True).view(-1)
mom_trans = torch.tensor([[0., -0.5], [0., 0.5]], requires_grad=True).view(-1)

gd_trans1 = torch.tensor([[-2., 0.], [2., 0.]], requires_grad=True).view(-1)
mom_trans1 = torch.tensor([[0., -0.5], [0., 0.5]], requires_grad=True).view(-1)

trans = im.DeformationModules.Translations(im.Manifolds.Landmarks(dim, nb_pts, gd=gd_trans, cotan=mom_trans), sigma)
trans1 = im.DeformationModules.Translations(im.Manifolds.Landmarks(dim, nb_pts, gd=gd_trans1, cotan=mom_trans1), sigma1)
hamiltonian = im.HamiltonianDynamic.Hamiltonian([trans, trans1])

intermediate_states, intermediate_controls = im.HamiltonianDynamic.shoot(hamiltonian, it, "torch_euler")

print("Initial")
print("trans")
print(gd_trans.view(-1, 2))
print(mom_trans.view(-1, 2))
print("trans1")
print(gd_trans1.view(-1, 2))
print(mom_trans1.view(-1, 2))

print("Final")
print("trans")
print(trans.manifold.gd.view(-1, 2))
print(trans.manifold.cotan.view(-1, 2))
print(trans.controls.view(-1, 2))
print("trans1")
print(trans1.manifold.gd.view(-1, 2))
print(trans1.manifold.cotan.view(-1, 2))
print(trans1.controls.view(-1, 2))

AssertionError: 

In [4]:
gd_trans = torch.tensor([[-2., 0.], [2., 0.]], requires_grad=True).view(-1)
mom_trans = torch.tensor([[0., -0.5], [0., 0.5]], requires_grad=True).view(-1)

gd_trans1 = torch.tensor([[-2., 0.], [2., 0.]], requires_grad=True).view(-1)
mom_trans1 = torch.tensor([[0., -0.5], [0., 0.5]], requires_grad=True).view(-1)

trans = im.DeformationModules.Translations(im.Manifolds.Landmarks(dim, nb_pts, gd=gd_trans, cotan=mom_trans), sigma)
trans1 = im.DeformationModules.Translations(im.Manifolds.Landmarks(dim, nb_pts, gd=gd_trans1, cotan=mom_trans1), sigma1)
hamiltonian = im.HamiltonianDynamic.Hamiltonian([trans, trans1])

intermediate_states_tdq, intermediate_controls_tdq = im.HamiltonianDynamic.shoot(hamiltonian, it, 'euler')
print("Initial")
print("trans")
print(gd_trans.view(-1, 2))
print(mom_trans.view(-1, 2))
print("trans1")
print(gd_trans1.view(-1, 2))
print(mom_trans1.view(-1, 2))

print("Final")
print("trans")
print(trans.manifold.gd.view(-1, 2))
print(trans.manifold.cotan.view(-1, 2))
print(trans.controls.view(-1, 2))
print("trans1")
print(trans1.manifold.gd.view(-1, 2))
print(trans1.manifold.cotan.view(-1, 2))
print(trans1.controls.view(-1, 2))

Initial
trans
tensor([[-2.,  0.],
        [ 2.,  0.]], grad_fn=<ViewBackward>)
tensor([[ 0.0000, -0.5000],
        [ 0.0000,  0.5000]], grad_fn=<ViewBackward>)
trans1
tensor([[-2.,  0.],
        [ 2.,  0.]], grad_fn=<ViewBackward>)
tensor([[ 0.0000, -0.5000],
        [ 0.0000,  0.5000]], grad_fn=<ViewBackward>)
Final
trans
tensor([[-1.8070, -1.7571],
        [ 1.8070,  1.7571]], grad_fn=<ViewBackward>)
tensor([[ 0.0978, -0.4682],
        [-0.0978,  0.4682]], grad_fn=<ViewBackward>)
tensor([[ 0.1866, -0.9442],
        [-0.1866,  0.9442]], grad_fn=<ViewBackward>)
trans1
tensor([[-1.8070, -1.7571],
        [ 1.8070,  1.7571]], grad_fn=<ViewBackward>)
tensor([[ 0.0978, -0.4682],
        [-0.0978,  0.4682]], grad_fn=<ViewBackward>)
tensor([[ 0.1866, -0.9442],
        [-0.1866,  0.9442]], grad_fn=<ViewBackward>)


In [12]:
# We compare intermediate states and controls of each shooting methods
for i in range(it):
    assert torch.allclose(intermediate_states[i][0].gd, intermediate_states_tdq[i][0].gd)
    assert torch.allclose(intermediate_states[i][0].cotan, intermediate_states_tdq[i][0].cotan)
    assert torch.allclose(intermediate_states[i][1].gd, intermediate_states_tdq[i][1].gd)
    assert torch.allclose(intermediate_states[i][1].cotan, intermediate_states_tdq[i][1].cotan)
    assert torch.allclose(intermediate_controls[i][0], intermediate_controls_tdq[i][0])
    assert torch.allclose(intermediate_controls[i][1], intermediate_controls_tdq[i][1])

In [ ]:
nx, ny = 100, 100
sx, sy = 10, 10
x, y = torch.meshgrid([torch.arange(0, nx), torch.arange(0, ny)])
x = sx*(x.type(torch.FloatTensor)/nx - 0.5)
y = sy*(y.type(torch.FloatTensor)/ny - 0.5)
u, v = dm.usefulfunctions.vec2grid(trans(dm.usefulfunctions.grid2vec(x, y).type(torch.FloatTensor)), nx, ny)

In [ ]:
%matplotlib qt5

plt.quiver(x.numpy(), y.numpy(), u.detach().numpy(), v.detach().numpy())
plt.show()